In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
data_dir = 'final_data'
out_dir = 'aggregated_data'
appname = 'LUBM-attribute'
#appname = 'LUBM-type'
#appname = 'LUBM_schemex'
#appname = 'dyldo_y2019_core_schemex'
#appname = 'dyldo_y2019_core_attribute'
#appname = 'dyldo_y2019_core_type'
#appname = 'dyldo_y2019_full_schemex'
#appname = 'dyldo_y2019_full_attribute'
#appname = 'dyldo_y2019_full_type'
appname = 'BSBM-schemex'
appname = 'BSBM-attribute'
appname = 'BSBM-type'
suffix = '-test-'

In [ ]:
export = True
# i = MIN; i < MAX
MIN = 1
MAX = 2 #4

track_stuff = ['performance.csv', 'changes.csv', 'update-time-and-space.csv']

for stuff in track_stuff:   
    with open(os.path.join(data_dir, appname + suffix + str(MIN) + '-' + stuff), 'r') as f:
        df = pd.read_csv(f, sep=',')
        df = df.set_index('Iteration')
        aggregated = df.copy()

        if MAX > MIN:
            # sum all values
            for t in range(MIN+1, MAX):
                with open(os.path.join(data_dir, appname + suffix + str(t)  + '-' + stuff), 'r') as f:
                    df = pd.read_csv(f, sep=',')


                for label, content in df.items():
                    if not label == 'Iteration':
                        for i in range(0, len(content)):
                            aggregated.at[i, label] = aggregated.at[i, label] + content[i]

            # divide all values
            for label, content in aggregated.items():
                if not label == 'Iteration':
                    for i in range(0, len(content)):
                        aggregated.at[i, label] = aggregated.at[i, label] / (MAX - MIN)

        display(aggregated)
        if export:
            if '-' in appname:
                aggregated.to_csv(out_dir + '/' + appname.replace('-', '_')  + '-' + stuff, index = 'Iteration', header=True)
            else:
                aggregated.to_csv(out_dir + '/' + appname  + '-' + stuff, index = 'Iteration', header=True)
  
        print('exported='+str(export))


In [ ]:
# check for average and deviation

export = True
# i = MIN; i < MAX
MIN = 1
MAX = 4 #4

apps = ['LUBM-attribute', 'LUBM-type', 'LUBM_schemex']
#, 'BSBM-schemex', 'BSBM-attribute','BSBM-type']
#        'dyldo_y2019_core_schemex', 'dyldo_y2019_core_attribute', 'dyldo_y2019_core_type']

#appname = 
#appname = 
#appname = 
#appname = 'dyldo_y2019_full_schemex'
#appname = 'dyldo_y2019_full_attribute'
#appname = 'dyldo_y2019_full_type'
#appname = 
#appname = 'BSBM-attribute'
#appname = 'BSBM-type'
suffix = '-test-'




stuff = 'performance.csv'
max_outliers = pd.DataFrame()

for appname in apps:
    run_times = pd.DataFrame(columns=['Iteration'])
    run_times = run_times.set_index('Iteration')

    orig_columns = []
    # get all values
    for t in range(MIN, MAX):
        try:
            with open(os.path.join(data_dir, appname + suffix + str(t)  + '-' + stuff), 'r') as f:
                df = pd.read_csv(f, sep=',')
                df = df.set_index('Iteration')
                if len(orig_columns) <= 0:
                    orig_columns = df.columns
                df.columns = list(map((lambda x: x+'_' + str(t)), df.columns))
                if run_times.empty:
                    run_times = df

                else:
                    run_times = run_times.merge(df, left_index=True, right_index=True)
        except FileNotFoundError:
            print(os.path.join(data_dir, appname + suffix + str(t)  + '-' + stuff))
            continue



    display(orig_columns)
    
    t_frame = {}    
    for col in orig_columns:
        get_cols = []
        for t in range(MIN, MAX): 
            get_cols.append(col+'_' + str(t))
        run_times[col+'_mean'] = run_times[get_cols].mean(axis=1)
        run_times[col+'_std'] = run_times[get_cols].std(axis=1)
        run_times[col+'_error'] = run_times[col+'_std'] / run_times[col+'_mean']

        t_frame[col] = run_times[col+'_error'].max()
    max_outliers = max_outliers.append(t_frame, ignore_index=True)
# if '-' in appname:
#     run_times.to_csv(out_dir + '/' + appname.replace('-', '_')  + '-run_times.csv', index = 'Iteration', header=True)
# else:
#     run_times.to_csv(out_dir + '/' + appname  + '-run_times.csv', index = 'Iteration', header=True)
  

display(max_outliers)
display(run_times)



In [ ]:
import matplotlib.pyplot as plt
import matplotlib
fig_width = 8
fig_height = 5

params = {
    'axes.labelsize': 12, # fontsize for x and y labels (was 10)
    'axes.titlesize': 12,
    #'text.fontsize': 8, # was 10
    'legend.fontsize': 12, # was 10
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'text.usetex': False,
    'figure.figsize': [fig_width,fig_height],
    'font.family': 'serif',
    'grid.linestyle': '--',
    'lines.linewidth': 2
}

matplotlib.rcParams.update(params)

test = run_times['Total_mean']

display(test.index)
x = np.array(test.index)
y = np.array(test) # Effectively y = x**2
e = np.array(run_times['Total_std'])

plt.errorbar(x, y, e, marker='^')





In [ ]:

matplotlib.rcParams.update(params)

test = run_times['Batch_mean']

display(test.index)
x = np.array(test.index)
y = np.array(test) # Effectively y = x**2
e = np.array(run_times['Batch_std'])

plt.errorbar(x, y, e, marker='^')